In [13]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

In [14]:
# dense Fully contected 
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons,
                 weight_regularizer_l1=0, weight_regularizer_l2=0,
                 bias_regularizer_l1=0, bias_regularizer_l2=0):

        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

        self.weight_regularizer_l1 = weight_regularizer_l1
        self.weight_regularizer_l2 = weight_regularizer_l2
        self.bias_regularizer_l1 = bias_regularizer_l1
        self.bias_regularizer_l2 = bias_regularizer_l2


    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.dot(inputs, self.weights) + self.biases

    def backward(self, dvalues):
        self.dweights = np.dot(self.inputs.T, dvalues)
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
    # Regulization 
        # Regularization Gradients
        #L1 → sparsity
        # L2 → smooth weights
        # Added directly to gradients (correct method)
        if self.weight_regularizer_l1 > 0:
            dL1 = np.ones_like(self.weights)
            dL1[self.weights < 0] = -1
            self.dweights += self.weight_regularizer_l1 * dL1

        if self.weight_regularizer_l2 > 0:
            self.dweights += 2 * self.weight_regularizer_l2 * self.weights

        if self.bias_regularizer_l1 > 0:
            dL1 = np.ones_like(self.biases)
            dL1[self.biases < 0] = -1
            self.dbiases += self.bias_regularizer_l1 * dL1

        if self.bias_regularizer_l2 > 0:
            self.dbiases += 2 * self.bias_regularizer_l2 * self.biases

        #Gradient Flow to Previous Layer
        self.dinputs = np.dot(dvalues, self.weights.T)


In [15]:
# ReLU Activation

In [16]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)
    def backward(self, dvalues):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0


In [17]:
# Sigmoid Activation

In [18]:
class Activation_Sigmoid:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = 1 / (1 + np.exp(-inputs))
    def backward(self, dvalues):
        self.dinputs = dvalues * self.output * (1 - self.output)


In [19]:
# Binary cross entropy  loss 

In [20]:
class Loss:
    def regularization_loss(self, layer):
        reg_loss = 0

        if layer.weight_regularizer_l1 > 0:
            reg_loss += layer.weight_regularizer_l1 * np.sum(np.abs(layer.weights))

        if layer.weight_regularizer_l2 > 0:
            reg_loss += layer.weight_regularizer_l2 * np.sum(layer.weights ** 2)

        if layer.bias_regularizer_l1 > 0:
            reg_loss += layer.bias_regularizer_l1 * np.sum(np.abs(layer.biases))

        if layer.bias_regularizer_l2 > 0:
            reg_loss += layer.bias_regularizer_l2 * np.sum(layer.biases ** 2)

        return reg_loss

class Loss_BinaryCrossentropy(Loss):
    def forward(self, y_pred, y_true):
        y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)

        sample_losses = -(y_true * np.log(y_pred) +
                          (1 - y_true) * np.log(1 - y_pred))

        return np.mean(sample_losses, axis=1)

    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        outputs = len(dvalues[0])

        clipped = np.clip(dvalues, 1e-7, 1 - 1e-7)

        self.dinputs = -(y_true / clipped -
                          (1 - y_true) / (1 - clipped)) / outputs

        self.dinputs /= samples


In [21]:
# Adam Optimizer 

In [22]:
class Optimizer_Adam:
    def __init__(self, learning_rate=0.001, decay=0., epsilon=1e-7,
                 beta_1=0.9, beta_2=0.999):

        self.learning_rate = learning_rate
        self.current_learning_rate = learning_rate
        self.decay = decay
        self.epsilon = epsilon
        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.iterations = 0

    def pre_update_params(self):
        if self.decay:
            self.current_learning_rate = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))
    def update_params(self, layer):
        if not hasattr(layer, 'weight_cache'):
            layer.weight_momentums = np.zeros_like(layer.weights)
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_momentums = np.zeros_like(layer.biases)
            layer.bias_cache = np.zeros_like(layer.biases)

        layer.weight_momentums = self.beta_1 * layer.weight_momentums + \
            (1 - self.beta_1) * layer.dweights
        layer.bias_momentums = self.beta_1 * layer.bias_momentums + \
            (1 - self.beta_1) * layer.dbiases

        layer.weight_cache = self.beta_2 * layer.weight_cache + \
            (1 - self.beta_2) * layer.dweights**2
        layer.bias_cache = self.beta_2 * layer.bias_cache + \
            (1 - self.beta_2) * layer.dbiases**2
        weight_c = layer.weight_cache / \
            (1 - self.beta_2 ** (self.iterations + 1))
        bias_c = layer.bias_cache / \
            (1 - self.beta_2 ** (self.iterations + 1))
        layer.weights += -self.current_learning_rate * \
            weight_m / (np.sqrt(weight_c) + self.epsilon)
        layer.biases += -self.current_learning_rate * \
            bias_m / (np.sqrt(bias_c) + self.epsilon)
    def post_update_params(self):
        self.iterations += 1


In [23]:
# Prepareing Dataset Non linearlly separable Data 

In [24]:
X, y = spiral_data(samples=100, classes=2)
y = y.reshape(-1, 1)

In [25]:
dense1 = Layer_Dense(2, 64, weight_regularizer_l2=5e-4, bias_regularizer_l2=5e-4)
activation1 = Activation_ReLU()

dense2 = Layer_Dense(64, 1)
activation2 = Activation_Sigmoid()

loss_function = Loss_BinaryCrossentropy()
optimizer = Optimizer_Adam(decay=5e-7)

In [26]:
# Training loop

In [27]:
for epoch in range(10001):

    dense1.forward(X)
    activation1.forward(dense1.output)

    dense2.forward(activation1.output)
    activation2.forward(dense2.output)
data_loss = loss_function.forward(activation2.output, y).mean()
reg_loss = loss_function.regularization_loss(dense1) + \
               loss_function.regularization_loss(dense2)
loss = data_loss + reg_loss


In [29]:
predictions = (activation2.output > 0.5) * 1
accuracy = np.mean(predictions == y)


In [34]:
if epoch % 100 == 0:
    print(f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}")
    loss_function.backward(activation2.output, y)
    activation2.backward(loss_function.dinputs)
    dense2.backward(activation2.dinputs)

    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)


epoch: 10000, acc: 0.500, loss: 0.693


In [35]:
X_test, y_test = spiral_data(samples=100, classes=2)
y_test = y_test.reshape(-1, 1)